# ASSOCIATION RULE LEARNING

📌 The steps to create a association rule learning are as follows:

  * Import Dataset

  * Data Preprocessing

  * Preparing ARL Data Structure (Invoice-Product Matrix)

  * Application: Association Rule Learning Recommender

# Import Necessary Libraries

In [1]:
!pip install mlxtend
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.width",500)
from mlxtend.frequent_patterns import apriori, association_rules

import warnings
warnings.filterwarnings('ignore')

## Import Dataset

If you want to download the dataset, you can use this <a href="https://archive.ics.uci.edu/ml/datasets/Online+Retail+II">link</a>

In [2]:
df_ = pd.read_excel("online_retail_II.xlsx",sheet_name="Year 2010-2011")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541910.0,9.552234,218.080957,-80995.00,1.00,3.00,10.00,80995.0
Price,541910.0,4.611138,96.759765,-11062.06,1.25,2.08,4.13,38970.0
Customer ID,406830.0,15287.684160,1713.603074,12346.00,13953.00,15152.00,16791.00,18287.0


## Data Preprocessing

In [4]:
def outlier_thresholds(dataframe,variable):
  quartile1 = dataframe[variable].quantile(0.01)
  quartile3 = dataframe[variable].quantile(0.99)
  interquartile_range = quartile3 - quartile1
  up_limit = quartile3 + 1.5 * interquartile_range
  low_limit = quartile1 - 1.5 * interquartile_range
  return low_limit,up_limit

In [5]:
def replace_with_thresholds(dataframe,variable):
  low_limit,up_limit = outlier_thresholds(dataframe,variable)
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
  dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [6]:
def data_prep(dataframe):
  dataframe.dropna(inplace=True)
  dataframe = dataframe[~dataframe["Invoice"].str.contains("C",na=False)]
  dataframe = dataframe[dataframe["Quantity"] > 0]
  dataframe = dataframe[dataframe["Price"] > 0]
  replace_with_thresholds(dataframe,"Quantity")
  replace_with_thresholds(dataframe,"Price")
  return dataframe

In [7]:
df = data_prep(df)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397885.0,11.830770,25.523052,1.000,2.00,6.00,12.00,298.50
Price,397885.0,2.893492,3.227175,0.001,1.25,1.95,3.75,37.06
Customer ID,397885.0,15294.416882,1713.144421,12346.000,13969.00,15159.00,16795.00,18287.00


## Preparing ARL Data Structure (Invoice-Product Matrix)

Burada amacımız satırlarda işlemler(faturalar-trasaction-sepet vs...) ve sutünlerde  ürünlerin olup olmama durumunu 0 ve 1 ile belirtiyoruz.

In [8]:
df_fr = df[df["Country"]=="France"] # Burada fransa ülkesinin birliktelik kurallarını üreteceğiz.
df_fr.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24.0,2010-12-01 08:45:00,3.75,12583.0,France
27,536370,22727,ALARM CLOCK BAKELIKE RED,24.0,2010-12-01 08:45:00,3.75,12583.0,France
28,536370,22726,ALARM CLOCK BAKELIKE GREEN,12.0,2010-12-01 08:45:00,3.75,12583.0,France
29,536370,21724,PANDA AND BUNNIES STICKER SHEET,12.0,2010-12-01 08:45:00,0.85,12583.0,France
30,536370,21883,STARS GIFT TAPE,24.0,2010-12-01 08:45:00,0.65,12583.0,France


Örnek olarak şöyle bir sernaryo düşünün: isveç pazarına firma olarak giriş yapacaksınız ancak sizin daha önce isveç müşterileriniz yok ve onlara ne önereceğinizi bilmiyorsunuz. Ancak isveçe en yakın ülke davranışı diyelim ki fransa olsun o zaman fransa ülkesinin birliktelik kurallarını çıkartacaksınız ve fransadaki müşterilerinizin aldıklarını belli bir işlemler ile isveçteki müşterilerinize önermiş olacaksınız.

In [9]:
# Fatura numarasına ve descritiona göre her üründen kaçar tane alınmış onu hesapladık.
df_fr.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).head()

Quantity
Invoice Description                               
536370   SET 2 TEA TOWELS I LOVE LONDON       24.0
        ALARM CLOCK BAKELIKE GREEN            12.0
        ALARM CLOCK BAKELIKE PINK             24.0
        ALARM CLOCK BAKELIKE RED              24.0
        CHARLOTTE BAG DOLLY GIRL DESIGN       20.0

In [10]:
# Ancak bizim isteğimiz her bir fatura için sutünlarda hangi üründen kaçar tane alınmasını yazsın
df_fr.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).unstack().iloc[0:5, 0:5]

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                 NaN                NaN                          NaN                      NaN                             24.0
536852                                 NaN                NaN                          NaN                      NaN                              NaN
536974                                 NaN                NaN                          NaN                      NaN                              NaN
537065                                 NaN                NaN                          NaN                      NaN                              NaN
537463                                 NaN                NaN                          NaN                      NaN                              NaN

In [11]:
# Şimdi burada isteğimiz ürünün olduğu yere 1 olmadığı yere 0 yazsın ve NaN kısımlardan kurtulalım
df_fr.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).unstack().fillna(0).iloc[0:5, 0:5]


Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                 0.0                0.0                          0.0                      0.0                             24.0
536852                                 0.0                0.0                          0.0                      0.0                              0.0
536974                                 0.0                0.0                          0.0                      0.0                              0.0
537065                                 0.0                0.0                          0.0                      0.0                              0.0
537463                                 0.0                0.0                          0.0                      0.0                              0.0

In [12]:
# Şimdi ise amacımıza tam olarak ulaştık. 0 ve 1 ile matris oluşturduk
df_fr.groupby(["Invoice", "Description"]).agg({"Quantity": "sum"}).unstack().fillna(0). \
applymap(lambda x: 1 if x>0 else 0).iloc[0:5, 0:5]


Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                   0                  0                            0                        0                                1
536852                                   0                  0                            0                        0                                0
536974                                   0                  0                            0                        0                                0
537065                                   0                  0                            0                        0                                0
537463                                   0                  0                            0                        0                                0

In [13]:
# Description'lar çok kafa karıştırıcı olduğu için StockeCode'ları yansıtmak daha iyidir.
df_fr.groupby(["Invoice", "StockCode"]).agg({"Quantity": "sum"}).unstack().fillna(0). \
applymap(lambda x: 1 if x>0 else 0).iloc[0:5, 0:5]

Quantity                        
StockCode    10002 10120 10125 10135 11001
Invoice                                   
536370           1     0     0     0     0
536852           0     0     0     0     0
536974           0     0     0     0     0
537065           0     0     0     0     0
537463           0     0     0     0     0

Şimdi yukarıda yaptıklarımızı fonksiyon ile yazalım

In [14]:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)

In [15]:
df_fr = df[df["Country"]=="France"]
fr_inv_pro_df = create_invoice_product_df(df_fr,id=True)
fr_inv_pro_df.iloc[0:5,0:20]

StockCode,10002,10120,10125,10135,11001,15036,15039,16012,16048,16218,16219,16225,16236,16237,16238,17174,20615,20617,20658,20665
Invoice,,,,,,,,,,,,,,,,,,,,
536370,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
537463,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Şimdi diyelim ki StockeCode ile mtrisi oluşturduğunuz ancak ihtiyaca göre hangi kodun hangi ürün olduğunu öğrenmek isterseniz aşağıdaki gibi yapabiliriz

In [16]:
df_fr[df_fr["StockCode"]==10120][["Description"]]

,Description
440034,DOGGY RUBBER


In [17]:
df_fr[df_fr["StockCode"]==10120][["Description"]].values[0].tolist()

['DOGGY RUBBER']

In [18]:
# Şimdi fonksiyonlaştıralım süreci
def check_id(dataframe, stock_code):
  product_name = dataframe[dataframe["StockCode"]==stock_code][["Description"]].values[0].tolist()
  print(product_name)

In [19]:
check_id(df_fr,10120)

['DOGGY RUBBER']


In [20]:
# Birliktelik kurallarının çıkarılmasının ilk adımı: Support değerlerinin hesaplanması
frequent_itemsets = apriori(fr_inv_pro_df, min_support=0.01, use_colnames=True)

In [22]:
frequent_itemsets.head()

,support,itemsets
0,0.020566,(10002)
1,0.015424,(10125)
2,0.010283,(16236)
3,0.012853,(16237)
4,0.012853,(16238)


In [23]:
# Support değerlerine göre confidence ve lift hesaplamak
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

In [25]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(10002),(21791),0.020566,0.028278,0.010283,0.500000,17.681818,0.009701,1.943445,0.963255
1,(21791),(10002),0.028278,0.020566,0.010283,0.363636,17.681818,0.009701,1.539111,0.970899
2,(10002),(21915),0.020566,0.069409,0.010283,0.500000,7.203704,0.008855,1.861183,0.879265
3,(21915),(10002),0.069409,0.020566,0.010283,0.148148,7.203704,0.008855,1.149771,0.925414
4,(10002),(22551),0.020566,0.136247,0.010283,0.500000,3.669811,0.007481,1.727506,0.742782


**antecedents**: İlk (Önceki) ürün anlamındadır.

**consequents**: İkini ürün anlamındadır.

**antecedent support**: İlk ürünün tek başına gözlenme olasılığı anlamındadır.

**consequent support**: İkinci ürünün tek başına gözlenme olasılığı anlamındadır.

**support**: antecedents ve consequents ürünlerinin birlikte gözlenme olasılığı anlamındadır.

**confidence**: X ürün alındığında Y ürünün alınma olasılığı anlamındadır.

**lift**: X ürün alındığında Y ürünün alınma olasılığı lift kadar artar.

**leverage**: lift değerine benzerdir ancak leverage, support değeri yüksek olanlara öncelik verir ve bundan dolayı ufak yanlılık payı vardır. o yüzden bir çalışmalarda lift değerini kullanmayı tercih ediyoruz çok yanlılık payı çok çok azdır.

**conviction**: Y ürünü olmadan X ürünün beklenen frekansıdır yada X ürünü olmadan Y ürünün beklenen frekansıdır.

**zhangs_metric**: X ürünün satın alındığında Y ürünün satın alınmama olasılığıdır.

Şimdi süreci tek bir fonksiyon ile yazmak

In [26]:
def create_rules(dataframe, id=True, country="France"):
  dataframe = dataframe[dataframe['Country'] == country]
  dataframe = create_invoice_product_df(dataframe, id)
  frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True)
  rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
  return rules

In [27]:
rules = create_rules(df)

In [28]:
rules[(rules["support"]>0.05) & (rules["confidence"]>0.1) & (rules["lift"]>5)].sort_values("confidence",ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
23707,"(21080, 21094)",(21086),0.102828,0.138817,0.100257,0.975000,7.023611,0.085983,34.447301,0.955918
23706,"(21080, 21086)",(21094),0.102828,0.128535,0.100257,0.975000,7.585500,0.087040,34.858612,0.967673
108821,"(21080, 21086, POST)",(21094),0.084833,0.128535,0.082262,0.969697,7.544242,0.071358,28.758355,0.947858
108822,"(21080, 21094, POST)",(21086),0.084833,0.138817,0.082262,0.969697,6.985410,0.070486,28.419023,0.936271
1777,(21094),(21086),0.128535,0.138817,0.123393,0.960000,6.915556,0.105550,21.529563,0.981563


Şimdi yukarıda gördüğünüz üzere 21080 kodlu ve 21094 kodulu ürün alındığında 21086 kodlu ürünün satın alınma olasılığı (confidence) 97% artıyor. ancak bu ürün kodlarının isimlerini öğrenmek için daha önce yazdığımız check_id metodu ile aşağıdaki gibi öğrenebiliriz.

In [32]:
check_id(df, 21080)
check_id(df, 21094)
check_id(df, 21086)

['SET/20 RED RETROSPOT PAPER NAPKINS ']
['SET/6 RED SPOTTY PAPER PLATES']
['SET/6 RED SPOTTY PAPER CUPS']


## Application: Association Rule Learning Recommender

İlk önce şunu söylemek lazım ki birliktelik kuralları, kullanıcı sepetine herhangi bir ürün eklemeden sistemde oluşturulur ve bir veri tabanında kaydedilir. Daha sonrasında kullanıcı ürün ekledikten sonra o ürüne göre hangi ürünü önereceğini veri tabnından okur ve kullanıcıya ürünü önerir.

Şimdi burada kullanıcı diyelim ki 21094 kodlu ürünü sepetine ekledi peki şimdi biz hangi ürünü ona önermeliyiz.

In [36]:
product_id = 21094 # kullanıcın sepetine eklendiği ürünün kodu
check_id(df, 21094) # kullanıcın sepetine eklendiği ürünün ismi

['SET/6 RED SPOTTY PAPER PLATES']


In [34]:
sorted_rules = rules.sort_values("lift", ascending=False) # öncelikle lift değerine göre sıralama yaparız.

In [39]:
recommendation_list = []
for i, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
        if j == product_id:
            recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

In [40]:
recommendation_list[0:2] # product_id kodlu ürünümüze iki tane ayrı ürün önerdik.

[21086, 22729]

Şimdi süreci fonksiyon halinde yazalım

In [41]:
def arl_recommender(rules_df, product_id, rec_count=1):
  sorted_rules = rules_df.sort_values("lift", ascending=False)
  recommendation_list = []
  for i, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
      if j == product_id:
        recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])
  return recommendation_list[0:rec_count]

In [42]:
arl_recommender(rules, 21094, 2)

[21086, 22729]

21094 kodlu ürün için önerilen diğer iki ürünün ismini öğrenmek içinde daha önce yazdığımız fonksiyonu kullanarak neler önerildiğini metinsel olarakta görebiliriz

In [43]:
check_id(df, 21094)
check_id(df, 21086)
check_id(df, 22729)

['SET/6 RED SPOTTY PAPER PLATES']
['SET/6 RED SPOTTY PAPER CUPS']
['ALARM CLOCK BAKELIKE ORANGE']
